# 主成分分析と自己符号化器の比較


主成分分析と自己符号化器が同一視されることがあるのは、次元削減や特徴抽出を自動で行うという共通の目的を持っているためです。両者は異なる手法ですが、いくつかの観点で類似している点があります。

#### 次元削減の目的: 
主成分分析と自己符号化器は、元のデータの次元を削減し、データの重要な情報を保持することを目指しています。どちらも高次元のデータを低次元の表現に変換することで、データの特徴をより効率的に表現することを目的としています。

#### 特徴の抽出: 
主成分分析と自己符号化器は、データの重要な特徴を抽出するために学習されます。主成分分析では、変数間の共分散行列から得られる固有ベクトルが主成分となり、データの情報を保持するように選択されます。自己符号化器では、データの再構成誤差を最小化するために、データの特徴を抽出するためのエンコーダとデコーダが学習されます。

#### 線形性の制約: 
主成分分析は線形な変換手法であり、変換後の主成分は元のデータの直交座標系の軸上に位置します。一方、自己符号化器は非線形な変換手法ですが、特に1層の自己符号化器の場合は線形変換に近い結果を得ることがあります。

以上の点から、主成分分析と自己符号化器は次元削減や特徴抽出という共通の目的を持ち、データの重要な情報を抽出するために使用されることが多いため、一部では同一視されることがあります。ただし、両者は異なる手法であり、結果が必ずしも一致しないこともあることに留意する必要があります。

In [4]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model

## 1. 主成分分析

主成分分析（Principal Component Analysis, PCA）は、多変量データの次元削減や特徴抽出によく用いられる統計手法です。

#### Step 1: 平均の算出
各変数の平均値を計算します。これにより、データセットの中心を原点に持ってくることができます。

#### Step 2: データの中心化
各変数からその変数の平均値を引くことで、データを中心化します。

#### Step 3: 共分散行列の計算
中心化されたデータセットから、変数間の共分散を表す共分散行列を計算します。

共分散行列は、データセットの転置行列とデータセット自体の積で得られます。共分散行列は正方行列であり、対角線上の要素は各変数の分散を表し、非対角線上の要素は変数間の共分散を表します。

#### Step 4: 共分散行列の固有値分解
共分散行列を固有値分解することで、主成分を得ることができます。固有値はデータセットの分散を表し、固有ベクトルはそれぞれの主成分軸を表します。

固有値分解によって、共分散行列から固有値と固有ベクトルを求めます。固有値は大きい順に並べ、固有ベクトルもそれに対応するように並べます。

#### Step 5: 主成分の選択
固有値の大きさが主成分の重要性を示しています。固有値が大きい順に固有ベクトルを選び、主成分とします。

#### Step 6: 主成分スコアの計算
主成分スコアは、各サンプル（観測値）を主成分軸上に射影した値です。これにより、元の多次元データを主成分の次元に射影したデータを得ることができます。

主成分スコアは、データセットの各サンプルと主成分ベクトルの内積で計算されます。

In [5]:
# サンプルデータの作成
'''
もともとは、4次元のデータを5つ作成している。
今から実行する主成分分析では，4次元のデータを2次元に削減する。
'''
data = np.array([[2, 3, 4, 5],
                 [1, 2, 3, 4],
                 [5, 5, 6, 7],
                 [6, 7, 8, 9],
                 [8, 9, 10, 11]])

# データの標準化
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 主成分分析の実行
pca = PCA(n_components=2)  # 次元削減後の次元数を指定
principal_components = pca.fit_transform(data_scaled)

# 結果の表示
print("Principal Components:")
print(principal_components)

Principal Components:
[[-1.75395186e+00 -6.73601671e-02]
 [-2.53364377e+00 -6.73788823e-02]
 [-1.41980226e-03  2.69272231e-01]
 [ 1.36481580e+00 -6.72853062e-02]
 [ 2.92419963e+00 -6.72478757e-02]]


## 2. 自己符号化器

自己符号化器（Autoencoder）は、ニューラルネットワークを用いた非線形次元削減手法です。一般的には、エンコーダとデコーダの2つの部分から構成されます。エンコーダは入力データを低次元の表現（潜在空間）に変換し、デコーダは潜在空間から元の次元に戻すように学習します。

ここでは、単純な1層の自己符号化器を例にして解説します。

#### Step 1: データの準備
解析対象のデータセットを準備します。例として、4次元のデータを持つデータセットを考えましょう。このデータセットは5つのサンプル（観測値）からなる行列とします。

#### Step 2: ネットワークの構築
自己符号化器のネットワークを構築します。単純な1層の自己符号化器では、エンコーダとデコーダの両方に1つの全結合層を使用します。エンコーダの出力次元は低次元の潜在空間の次元となります。

#### Step 3: ネットワークの学習
データセットを用いて自己符号化器のネットワークを学習させます。学習の目標は、デコーダの出力が元のデータに近づくようにすることです。

#### Step 4: エンコーダの活性化値の取得
学習が終了したら、エンコーダの出力層の活性化値（潜在空間の表現）を取得します。この潜在空間の表現が次元削減や特徴抽出の結果となります。

#### Step 5: デコーダを用いた再構成
潜在空間の表現を入力として、デコーダを使って元の次元に再構成します。デコーダの出力は元のデータセットの再構成となります。

以上が自己符号化器の手順です。エンコーダの出力が潜在空間の表現であり、デコーダを用いて元の次元に再構成することで、次元削減や特徴抽出が行われます。自己符号化器は非線形な変換を学習することができるため、主成分分析よりも柔軟な次元削減が可能となります。

In [6]:
'''
主成分分析と同様に，自己符号化器も次元削減を行うことができる。
今回は，4次元のデータを2次元に削減する。
'''
# サンプルデータの作成
data = np.array([[2, 3, 4, 5],
                 [1, 2, 3, 4],
                 [5, 5, 6, 7],
                 [6, 7, 8, 9],
                 [8, 9, 10, 11]])

# データの標準化
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 自己符号化器の構築
input_dim = data.shape[1]  # 入力次元数
encoding_dim = 2  # 潜在空間の次元数

# エンコーダの定義
input_data = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_data)

# エンコーダのモデルの定義
encoder = Model(input_data, encoded)

# エンコーダの出力を取得
encoded_data = encoder.predict(data_scaled)

# 結果の表示
print("Encoded Data:")
print(encoded_data)


1/1 [==============================] - 0s 62ms/step
Encoded Data:
[[0.         0.        ]
 [0.         0.        ]
 [0.1399063  0.05154558]
 [1.0027106  0.16967405]
 [2.1886387  0.3783146 ]]


また，損失関数に正則化項を追加した学習を施すことも可能です．この場合，潜在空間の表現がスパースになるように学習されます．

In [7]:
from keras import regularizers
import numpy as np

# サンプルデータの作成
data = np.array([[2, 3, 4, 5],
                 [1, 2, 3, 4],
                 [5, 5, 6, 7],
                 [6, 7, 8, 9],
                 [8, 9, 10, 11]])

# データの標準化
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 自己符号化器の構築
input_dim = data.shape[1]  # 入力次元数
encoding_dim = 2  # 潜在空間の次元数

# エンコーダの定義
input_data = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(0.01))(input_data)

# デコーダの定義
decoded = Dense(input_dim, activation='linear')(encoded)

# 自己符号化器のモデルの定義
autoencoder = Model(input_data, decoded)

# 自己符号化器のコンパイルと学習
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(data_scaled, data_scaled, epochs=100, batch_size=1)

# エンコーダの出力を取得
encoder = Model(input_data, encoded)
encoded_data = encoder.predict(data_scaled)

# 結果の表示
print("Encoded Data:")
print(encoded_data)

Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 2.9990
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 2.9728
Epoch 3/100
5/5 [==============================] - 0s 969us/step - loss: 2.9164
Epoch 4/100
5/5 [==============================] - 0s 1ms/step - loss: 2.8845
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 2.8405
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 2.8141
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 2.7681
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 2.7508
Epoch 9/100
5/5 [==============================] - 0s 1ms/step - loss: 2.7009
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 2.6776
Epoch 11/100
5/5 [==============================] - 0s 1ms/step - loss: 2.6364
Epoch 12/100
5/5 [==============================] - 0s 1ms/step - loss: 2.5998
Epoch 13/100
5/5 [==============================] - 0s 2ms/